In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from moSlicer import MoSlicer
import moMetrics as MoMetrics

In [2]:
# Set up MoSlicer. 
orbitfile = 'pha20141031.des'
obsfile = 'test_out.txt'

In [8]:
mos = MoSlicer(orbitfile, Hrange=np.arange(13, 25, 0.5))
mos.readObs(obsfile)
print mos.ssoObs['objId'][0]
print mos.orbits.dtype.names

0
('objId', 'q', 'e', 'inc', 'node', 'argPeri', 'tPeri', 'epoch', 'H', 'g', 'a', 'meanAnom')


In [6]:
for i, slicevals in enumerate(mos):
    print i, len(slicevals['obs']), slicevals['Hvals'], slicevals['obs']['magFilter']

0
0 2024 [ 13.   13.5  14.   14.5  15.   15.5  16.   16.5  17.   17.5  18.   18.5
  19.   19.5  20.   20.5  21.   21.5  22.   22.5  23.   23.5  24.   24.5] ['20.9694487512-0.291351785733' '20.9694229-0.291351785733'
 '21.1868442003-0.298075537869' ..., '21.5306427033-0.291351785733'
 '23.5022602782-0.298075537869' '23.5026402409-0.298075537869']
1
1 264 [ 13.   13.5  14.   14.5  15.   15.5  16.   16.5  17.   17.5  18.   18.5
  19.   19.5  20.   20.5  21.   21.5  22.   22.5  23.   23.5  24.   24.5] ['18.3645859409-0.298075537869' '18.3657499931-0.298075537869'
 '18.3662377051-0.298075537869' '18.8996559424-0.298075537869'
 '18.9006306078-0.298075537869' '19.7383543714-0.30256349639'
 '19.8465905482-0.298075537869' '19.9013312572-0.298075537869'
 '19.9021029087-0.298075537869' '20.4999038133-0.30256349639'
 '20.6276133537-0.30256349639' '20.6276748234-0.30256349639'
 '21.4395868139-0.30256349639' '21.56404580721.6144033523'
 '22.0423154396-0.298075537869' '22.0427062216-0.298075537869'
 

In [9]:
metricVals = np.empty(mos.slicerShape, dtype='float')
metric = MoMetrics.DiscoveryMetric()

In [17]:
for i, slicevals in enumerate(mos):
    for j, Hval in enumerate(slicevals['Hvals']):
        print Hval, slicevals['orb']['H'], slicevals['obs']['magFilter']
        metricVals[i][j] = metric.run(slicevals['obs'], Hval, slicevals['orb']['H'])

13.0 20.0 ['20.9694487512-0.291351785733' '20.9694229-0.291351785733'
 '21.1868442003-0.298075537869' ..., '21.5306427033-0.291351785733'
 '23.5022602782-0.298075537869' '23.5026402409-0.298075537869']


TypeError: unsupported operand type(s) for +: 'float' and 'str'